In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()



c:\Users\91800\Desktop\GENAI UDEMY\genai-udemy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\91800\Desktop\GENAI UDEMY\genai-udemy\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [2]:
class GemmaRouter:
    """
    A simple wrapper for using Google's Gemma-2-2B-IT model
    via Hugging Face's OpenAI-compatible inference API.
    """

    def __init__(
        self,
        model: str = "google/gemma-2-2b-it:nebius",
        token_env: str = "HF_THIRD_TOKEN",
        base_url: str = "https://router.huggingface.co/v1",
    ):
        api_key = os.getenv(token_env)
        if not api_key:
            raise ValueError(f"Missing token: env variable '{token_env}' not found.")

        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model = model

    def invoke(self, prompt: str, temperature: float = 0.7, max_tokens: int = 256):
        """Non-streaming response."""
        completion = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.choices[0].message.content.strip()


# Initialize Gemma model
model = GemmaRouter()



In [4]:
model2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [5]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the following properties from the passage:

- sentiment: The sentiment of the text
- political_tendency: The political tendency of the user
- language: The language the text is written in

Passage:
{input}

Return your answer in plain text or JSON.
"""
)



class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    political_tendency: str = Field(description="The political tendency of the user")
    language: str = Field(description="The language the text is written in")



In [26]:

trump_follower = "I'm confident that President Trump's leadership and track record will once again resonate with Americans. His strong stance on economic growth and national security is exactly what our country needs at this pivotal moment. We need to bring back the proven leadership that can make America great again!"

biden_follower = "I believe President Biden's compassionate and steady approach is vital for our nation right now. His commitment to healthcare reform, climate change, and restoring our international alliances is crucial. It's time to continue the progress and ensure a future that benefits all Americans.But id d"


itachi_follower="People see only the mask I wear, but not the wars I fight within. My silence is mistaken for coldness, when it is simply the only place where my broken pieces don’t tremble."

In [37]:
chain2= tagging_prompt | model2

In [39]:
chain2.invoke({"input":itachi_follower})

AIMessage(content='```json\n{\n  "sentiment": "Negative (sadness, vulnerability, loneliness)",\n  "political_tendency": "Not applicable (no political content)",\n  "language": "English"\n}\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--4421d2b4-fa0c-4a77-be5e-0ac5c8c929ef-0', usage_metadata={'input_tokens': 99, 'output_tokens': 47, 'total_tokens': 304, 'input_token_details': {'cache_read': 0}})

In [35]:
import json

def groq_call(prompt):
    response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

model3 = RunnableLambda(groq_call)

chain = (
    tagging_prompt
    | RunnableLambda(lambda x: x.to_string())   # FIX HERE
    | model3
)

chain.invoke({"input": itachi_follower})


'Here\'s the extracted information in JSON format:\n\n```json\n{\n  "sentiment": "negative/emotional",\n  "political_tendency": "neutral",\n  "language": "English"\n}\n```\n\nIn this passage, the sentiment is negative/emotional because the speaker mentions \'wars I fight within\' and \'broken pieces,\' indicating inner turmoil. They also express how their silence is misinterpreted, which adds to the emotional tone.\n\nAs for the political tendency, it is neutral. The passage does not convey any specific stance on politics or government, and the focus is on the speaker\'s personal emotions and inner struggles.\n\nThe language is English, which is clearly indicated by the use of English words and grammar throughout the passage.'